# Install dependencies, import modules, input credentials

In [ ]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import praw
from datetime import datetime
import logging
import time
from prawcore.exceptions import TooManyRequests
import pandas as pd
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client_id = ""
client_secret = ""

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="Scraper",
)

In [ ]:
# suppress warnings from PRAW
logging.getLogger('praw').setLevel(logging.ERROR)

# Merge submissions and comments dfs collected over multiple weeks

In [ ]:
# define functions
def merge_csv(csv_list) :
    merged_df = pd.DataFrame()

    for csv in csv_list :
        df = pd.read_csv(csv, index_col=0)
        print(f"{csv}: {df.shape}")
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    num_1 = merged_df.shape[0]

    # drop duplicates
    duplicated = merged_df['id'].duplicated()
    if duplicated.any() :
        print("\nThere are duplicates.")
        merged_df.drop_duplicates(subset='id')
    else :
        print("\nThere are no duplicates.")

    # drop rows where author is "AutoModerator" or author is "None"
    merged_df = merged_df[(merged_df['author'] != "None") & (merged_df['author'] != "AutoModerator")].reset_index(drop=True)

    num_2 = merged_df.shape[0]
    print(f"\nDropped {num_1 - num_2} rows.\n")

    return merged_df

In [ ]:
submissions = [
    '20231016_comm557_submissions_df.csv',
    '20231024_comm557_submissions_df.csv',
    '20231109_comm557_submissions_df.csv',
    '20231115_comm557_submissions_df.csv']
comments = [
    '20231016_comm557_comments_df.csv',
    '20231024_comm557_comments_df.csv',
    '20231109_comm557_comments_df.csv',
    '20231115_comm557_comments_df.csv']

In [ ]:
submissions_df = merge_csv(submissions)

# drop rows in submissions_df where num_comments is more than 1 (just to cut down on the dataset)
num_1 = submissions_df.shape[0]
submissions_df = submissions_df[submissions_df['num_comments'] >= 1].reset_index(drop=True)
num_2 = submissions_df.shape[0]
print(f"Dropped {num_1 - num_2} rows.\n")

print(f"Shape: {submissions_df.shape}")
submissions_df # 3064 // 1347

20231016_comm557_submissions_df.csv: (993, 15)
20231024_comm557_submissions_df.csv: (697, 15)
20231109_comm557_submissions_df.csv: (709, 15)
20231115_comm557_submissions_df.csv: (741, 15)

There are no duplicates.

Dropped 10 rows.

Dropped 66 rows.

Shape: (3064, 15)


,id,full_id,author,title,text,url,is_self,created_utc,edited,num_comments,is_locked,is_over_18,score,upvote_ratio,link
0,1793g27,t3_1793g27,jelindrael,What are books with only strategies on how to ...,"Basically, I am looking for a book that will j...",https://www.reddit.com/r/ADHD/comments/1793g27...,True,2023-10-16 11:06:54,False,2,False,False,1,1.00,/r/ADHD/comments/1793g27/what_are_books_with_o...
1,1793f20,t3_1793f20,Babybabyyyx,Teach me how to be quieter,I always feel bad when I talk too much regardl...,https://www.reddit.com/r/ADHD/comments/1793f20...,True,2023-10-16 11:05:06,False,1,False,False,1,1.00,/r/ADHD/comments/1793f20/teach_me_how_to_be_qu...
2,1793bb4,t3_1793bb4,Tip_Of_The_Sauce,Not sure where to go from here,My entire life I’ve been told that I have ADHD...,https://www.reddit.com/r/ADHD/comments/1793bb4...,True,2023-10-16 10:59:05,False,1,False,False,1,1.00,/r/ADHD/comments/1793bb4/not_sure_where_to_go_...
3,1793a88,t3_1793a88,Cursed_Creative,Forgetting that you remembered?,I don't know if this is a new phenomenon for m...,https://www.reddit.com/r/ADHD/comments/1793a88...,True,2023-10-16 10:56:56,False,1,False,False,1,1.00,/r/ADHD/comments/1793a88/forgetting_that_you_r...
4,17932t2,t3_17932t2,comingloose,what was the thing that finally helped you sto...,What’s been your “silver bullet” for getting a...,https://www.reddit.com/r/ADHD/comments/17932t2...,True,2023-10-16 10:43:04,False,1,False,False,2,1.00,/r/ADHD/comments/17932t2/what_was_the_thing_th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,17sgq12,t3_17sgq12,anonymousdumbhoe,ADHD + OCD oversharing = hellish combo..,I keep accidentally oversharing things and the...,https://www.reddit.com/r/ADHD/comments/17sgq12...,True,2023-11-10 23:17:04,2023-11-10 23:27:03,2,False,False,4,1.00,/r/ADHD/comments/17sgq12/adhd_ocd_oversharing_...
3060,17sgljq,t3_17sgljq,SPdoc,I’m often seen as defensive if I explain or co...,"First off, does anyone else have this problem ...",https://www.reddit.com/r/ADHD/comments/17sgljq...,True,2023-11-10 23:11:13,False,12,False,False,11,1.00,/r/ADHD/comments/17sgljq/im_often_seen_as_defe...
3061,17sg00i,t3_17sg00i,diamondaires,"My professor wants proof, who should I go to?",I asked for an extension on my assignments and...,https://www.reddit.com/r/ADHD/comments/17sg00i...,True,2023-11-10 22:43:35,False,39,False,False,7,0.83,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
3062,17sfvtx,t3_17sfvtx,suckadipdumbshitz,"Misdiagnosed with ADHD, How was I able to take...","As the title says, my old psychiatrist diagnos...",https://www.reddit.com/r/ADHD/comments/17sfvtx...,True,2023-11-10 22:38:10,False,4,False,False,2,1.00,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...


In [ ]:
comments_df = merge_csv(comments)

print(f"Shape: {comments_df.shape}")
comments_df # 43764

20231016_comm557_comments_df.csv: (10486, 13)
20231024_comm557_comments_df.csv: (12790, 13)
20231109_comm557_comments_df.csv: (10460, 13)
20231115_comm557_comments_df.csv: (10502, 13)

There are no duplicates.

Dropped 474 rows.

Shape: (43764, 13)


,id,author,body,created_utc,edited,distinguished,replies,submission_id,parent_id,is_submitter,is_stickied,score,link
0,k53n1ca,throwawaytheist,The Anti Planner\n\nIt's literally just a list...,2023-10-16 11:12:54,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
1,k53pelr,No-Sea1173,Don't know any books sorry. \n\nMaybe this wil...,2023-10-16 11:37:22,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
2,k53nmyv,Erpelpelle,"You can try to focus on your breathing, recogn...",2023-10-16 11:19:19,False,NaN,0,t3_1793f20,t3_1793f20,False,False,1,/r/ADHD/comments/1793f20/teach_me_how_to_be_qu...
3,k53o6hz,Cursed_Creative,"Not to discourage you, but just for N=1, my ex...",2023-10-16 11:24:57,False,NaN,0,t3_1792v8g,t3_1792v8g,False,False,1,/r/ADHD/comments/1792v8g/research_student_prod...
4,k53kt93,CaberTosser_,Yeah. Dopamine deficiency and depression can c...,2023-10-16 10:48:09,False,NaN,0,t3_1792p91,t3_1792p91,False,False,2,/r/ADHD/comments/1792p91/i_dont_remember_anyth...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43759,k8psrst,Benu5,">When I was being medicated, I was very anxiou...",2023-11-10 23:43:28,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...
43760,k8ru401,FuzzyAd9604,"You've got both, it's super common & or they h...",2023-11-11 11:44:50,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...
43761,k8qxjkx,ADHD-Fens,I would say anything that has you learning ski...,2023-11-11 05:04:59,False,NaN,1,t3_17sfu81,t3_17sfu81,False,False,3,/r/ADHD/comments/17sfu81/best_and_worst_therap...
43762,k8ucf2e,ThrowThinkAway,"This was insightful, thank you. The point abou...",2023-11-11 21:36:15,False,NaN,1,t3_17sfu81,t1_k8qxjkx,True,False,1,/r/ADHD/comments/17sfu81/best_and_worst_therap...


In [ ]:
# drop rows in comments_df that are replying to a submission not in submissions_df
submissions_id_df = submissions_df['full_id'].to_frame()
comments_df = comments_df[comments_df['submission_id'].isin(submissions_id_df['full_id'])].reset_index(drop=True)

comments_df # 43597 // 41127

,id,author,body,created_utc,edited,distinguished,replies,submission_id,parent_id,is_submitter,is_stickied,score,link
0,k53n1ca,throwawaytheist,The Anti Planner\n\nIt's literally just a list...,2023-10-16 11:12:54,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
1,k53pelr,No-Sea1173,Don't know any books sorry. \n\nMaybe this wil...,2023-10-16 11:37:22,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
2,k53nmyv,Erpelpelle,"You can try to focus on your breathing, recogn...",2023-10-16 11:19:19,False,NaN,0,t3_1793f20,t3_1793f20,False,False,1,/r/ADHD/comments/1793f20/teach_me_how_to_be_qu...
3,k53o6hz,Cursed_Creative,"Not to discourage you, but just for N=1, my ex...",2023-10-16 11:24:57,False,NaN,0,t3_1792v8g,t3_1792v8g,False,False,1,/r/ADHD/comments/1792v8g/research_student_prod...
4,k53kt93,CaberTosser_,Yeah. Dopamine deficiency and depression can c...,2023-10-16 10:48:09,False,NaN,0,t3_1792p91,t3_1792p91,False,False,2,/r/ADHD/comments/1792p91/i_dont_remember_anyth...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43592,k8psrst,Benu5,">When I was being medicated, I was very anxiou...",2023-11-10 23:43:28,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...
43593,k8ru401,FuzzyAd9604,"You've got both, it's super common & or they h...",2023-11-11 11:44:50,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...
43594,k8qxjkx,ADHD-Fens,I would say anything that has you learning ski...,2023-11-11 05:04:59,False,NaN,1,t3_17sfu81,t3_17sfu81,False,False,3,/r/ADHD/comments/17sfu81/best_and_worst_therap...
43595,k8ucf2e,ThrowThinkAway,"This was insightful, thank you. The point abou...",2023-11-11 21:36:15,False,NaN,1,t3_17sfu81,t1_k8qxjkx,True,False,1,/r/ADHD/comments/17sfu81/best_and_worst_therap...


In [ ]:
submissions_df.to_csv("/content/comm557_submissions_df.csv")

In [ ]:
comments_df.to_csv("/content/comm557_comments_df.csv")

# Creating network df

In [ ]:
# getting comments that are under other comments
comments_to_comments_df = comments_df[(comments_df.parent_id.str.startswith('t1'))]
comments_to_comments_df # 18251 // 17724

,id,author,body,created_utc,edited,distinguished,replies,submission_id,parent_id,is_submitter,is_stickied,score,link
10,k53hxgg,NOTeRcHAThiO,Don’t the first 6 override the 12 that follow?...,2023-10-16 10:13:10,False,NaN,0,t3_1791ysk,t1_k53fshg,True,False,1,/r/ADHD/comments/1791ysk/is_it_just_me_or_coul...
12,k53d8tr,ItsAleexB21,problem is that i've had the referral. i've ha...,2023-10-16 09:10:52,False,NaN,1,t3_1791d27,t1_k53c607,True,False,1,/r/ADHD/comments/1791d27/how_to_cope_with_adhd...
13,k53ed1w,Debaucherous-Me,Who was the referrals to? I psychiatrist could...,2023-10-16 09:26:06,False,NaN,1,t3_1791d27,t1_k53d8tr,False,False,1,/r/ADHD/comments/1791d27/how_to_cope_with_adhd...
14,k53fsmn,ItsAleexB21,they were two separate consultants. for anythi...,2023-10-16 09:45:37,False,NaN,1,t3_1791d27,t1_k53ed1w,True,False,1,/r/ADHD/comments/1791d27/how_to_cope_with_adhd...
15,k53g3ik,Debaucherous-Me,That system is bullshit! Once I had my referra...,2023-10-16 09:49:38,False,NaN,1,t3_1791d27,t1_k53fsmn,False,False,1,/r/ADHD/comments/1791d27/how_to_cope_with_adhd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43589,k8w90xt,diamondaires,The reality is I should’ve gotten tested and d...,2023-11-12 05:49:27,False,NaN,1,t3_17sg00i,t1_k8ri3c7,True,False,1,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
43590,k8w9ous,diamondaires,I’ve only been able to find instances where it...,2023-11-12 05:56:17,False,NaN,0,t3_17sg00i,t1_k8r3r2t,True,False,1,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
43591,k8y2r5g,fae713,I can completely understand the frustration of...,2023-11-12 16:51:42,False,NaN,0,t3_17sg00i,t1_k8w90xt,False,False,1,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
43595,k8ucf2e,ThrowThinkAway,"This was insightful, thank you. The point abou...",2023-11-11 21:36:15,False,NaN,1,t3_17sfu81,t1_k8qxjkx,True,False,1,/r/ADHD/comments/17sfu81/best_and_worst_therap...


In [ ]:
# getting comments that are directly under submissions & are not made by the submitter
comments_to_submissions_df = comments_df[(comments_df.is_submitter == False) & (comments_df.parent_id.str.startswith('t3'))]
comments_to_submissions_df # 25236 // 23329

,id,author,body,created_utc,edited,distinguished,replies,submission_id,parent_id,is_submitter,is_stickied,score,link
0,k53n1ca,throwawaytheist,The Anti Planner\n\nIt's literally just a list...,2023-10-16 11:12:54,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
1,k53pelr,No-Sea1173,Don't know any books sorry. \n\nMaybe this wil...,2023-10-16 11:37:22,False,NaN,0,t3_1793g27,t3_1793g27,False,False,1,/r/ADHD/comments/1793g27/what_are_books_with_o...
2,k53nmyv,Erpelpelle,"You can try to focus on your breathing, recogn...",2023-10-16 11:19:19,False,NaN,0,t3_1793f20,t3_1793f20,False,False,1,/r/ADHD/comments/1793f20/teach_me_how_to_be_qu...
3,k53o6hz,Cursed_Creative,"Not to discourage you, but just for N=1, my ex...",2023-10-16 11:24:57,False,NaN,0,t3_1792v8g,t3_1792v8g,False,False,1,/r/ADHD/comments/1792v8g/research_student_prod...
4,k53kt93,CaberTosser_,Yeah. Dopamine deficiency and depression can c...,2023-10-16 10:48:09,False,NaN,0,t3_1792p91,t3_1792p91,False,False,2,/r/ADHD/comments/1792p91/i_dont_remember_anyth...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43564,k8q4byd,ItsCoolDani,What did you initially say to the prof when as...,2023-11-11 01:07:28,False,NaN,0,t3_17sg00i,t3_17sg00i,False,False,1,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
43565,k8q97v5,Acti-Verse,Go to the office and ask for special accommoda...,2023-11-11 01:43:41,False,NaN,0,t3_17sg00i,t3_17sg00i,False,False,1,/r/ADHD/comments/17sg00i/my_professor_wants_pr...
43592,k8psrst,Benu5,">When I was being medicated, I was very anxiou...",2023-11-10 23:43:28,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...
43593,k8ru401,FuzzyAd9604,"You've got both, it's super common & or they h...",2023-11-11 11:44:50,False,NaN,0,t3_17sfvtx,t3_17sfvtx,False,False,1,/r/ADHD/comments/17sfvtx/misdiagnosed_with_adh...


In [ ]:
# comments to comments
df1 = comments_to_comments_df[['author', 'parent_id', 'body']].copy()
df1['parent_id'] = df1['parent_id'].str.split('_').str[-1]
df1.columns = ['comment_to_comment_author', 'comment_id', 'comment_body']
df1 # 18251

,comment_to_comment_author,comment_id,comment_body
10,NOTeRcHAThiO,k53fshg,Don’t the first 6 override the 12 that follow?...
12,ItsAleexB21,k53c607,problem is that i've had the referral. i've ha...
13,Debaucherous-Me,k53d8tr,Who was the referrals to? I psychiatrist could...
14,ItsAleexB21,k53ed1w,they were two separate consultants. for anythi...
15,Debaucherous-Me,k53fsmn,That system is bullshit! Once I had my referra...
...,...,...,...
43589,diamondaires,k8ri3c7,The reality is I should’ve gotten tested and d...
43590,diamondaires,k8r3r2t,I’ve only been able to find instances where it...
43591,fae713,k8w90xt,I can completely understand the frustration of...
43595,ThrowThinkAway,k8qxjkx,"This was insightful, thank you. The point abou..."


In [ ]:
df2 = comments_to_comments_df[['author', 'id']]
df2.columns = ['comment_author', 'comment_id']
df2 # 18251

,comment_author,comment_id
10,NOTeRcHAThiO,k53hxgg
12,ItsAleexB21,k53d8tr
13,Debaucherous-Me,k53ed1w
14,ItsAleexB21,k53fsmn
15,Debaucherous-Me,k53g3ik
...,...,...
43589,diamondaires,k8w90xt
43590,diamondaires,k8w9ous
43591,fae713,k8y2r5g
43595,ThrowThinkAway,k8ucf2e


In [ ]:
df = df1.merge(df2, how='inner', on='comment_id')
comments_network_df = df[['comment_to_comment_author', 'comment_author', 'comment_id', 'comment_body']]
comments_network_df.columns = ['source', 'target', 'target_id', 'comment']
comments_network_df # 9534

,source,target,target_id,comment
0,Debaucherous-Me,ItsAleexB21,k53d8tr,Who was the referrals to? I psychiatrist could...
1,ItsAleexB21,Debaucherous-Me,k53ed1w,they were two separate consultants. for anythi...
2,Debaucherous-Me,ItsAleexB21,k53fsmn,That system is bullshit! Once I had my referra...
3,ItsAleexB21,Debaucherous-Me,k53g3ik,tell me about it! yeah im thinking im going to...
4,MyFavouriteJo,axjunkie93,k53a0oi,Seriously?!! I’ve never heard of that happenin...
...,...,...,...,...
9529,diamondaires,fae713,k8rhqcs,"Yes, I know they’re not legally required to do..."
9530,diamondaires,fae713,k8ri3c7,The reality is I should’ve gotten tested and d...
9531,diamondaires,Embarrassed_Tie_9346,k8r3r2t,I’ve only been able to find instances where it...
9532,fae713,diamondaires,k8w90xt,I can completely understand the frustration of...


In [ ]:
# comments to submissions
df3 = comments_to_submissions_df[['author', 'body', 'submission_id']].copy()
df3['submission_id'] = df3['submission_id'].str.split('_').str[-1]
df3.columns = ['comment_author', 'comment_body', 'submission_id']
df3 # 25236

,comment_author,comment_body,submission_id
0,throwawaytheist,The Anti Planner\n\nIt's literally just a list...,1793g27
1,No-Sea1173,Don't know any books sorry. \n\nMaybe this wil...,1793g27
2,Erpelpelle,"You can try to focus on your breathing, recogn...",1793f20
3,Cursed_Creative,"Not to discourage you, but just for N=1, my ex...",1792v8g
4,CaberTosser_,Yeah. Dopamine deficiency and depression can c...,1792p91
...,...,...,...
43564,ItsCoolDani,What did you initially say to the prof when as...,17sg00i
43565,Acti-Verse,Go to the office and ask for special accommoda...,17sg00i
43592,Benu5,">When I was being medicated, I was very anxiou...",17sfvtx
43593,FuzzyAd9604,"You've got both, it's super common & or they h...",17sfvtx


In [ ]:
df4 = submissions_df[['author', 'id']]
df4.columns = ['submission_author', 'submission_id']
df4 # 3064

,submission_author,submission_id
0,jelindrael,1793g27
1,Babybabyyyx,1793f20
2,Tip_Of_The_Sauce,1793bb4
3,Cursed_Creative,1793a88
4,comingloose,17932t2
...,...,...
3059,anonymousdumbhoe,17sgq12
3060,SPdoc,17sgljq
3061,diamondaires,17sg00i
3062,suckadipdumbshitz,17sfvtx


In [ ]:
df = df3.merge(df4, how='inner', on='submission_id')
sub_com_network_df = df[['comment_author', 'submission_author','submission_id', 'comment_body']]
sub_com_network_df.columns = ['source', 'target', 'target_id', 'comment']
sub_com_network_df # 25236

,source,target,target_id,comment
0,throwawaytheist,jelindrael,1793g27,The Anti Planner\n\nIt's literally just a list...
1,No-Sea1173,jelindrael,1793g27,Don't know any books sorry. \n\nMaybe this wil...
2,Erpelpelle,Babybabyyyx,1793f20,"You can try to focus on your breathing, recogn..."
3,Cursed_Creative,YogurtclosetSoft3850,1792v8g,"Not to discourage you, but just for N=1, my ex..."
4,CaberTosser_,nilbogpt,1792p91,Yeah. Dopamine deficiency and depression can c...
...,...,...,...,...
25231,ItsCoolDani,diamondaires,17sg00i,What did you initially say to the prof when as...
25232,Acti-Verse,diamondaires,17sg00i,Go to the office and ask for special accommoda...
25233,Benu5,suckadipdumbshitz,17sfvtx,">When I was being medicated, I was very anxiou..."
25234,FuzzyAd9604,suckadipdumbshitz,17sfvtx,"You've got both, it's super common & or they h..."


In [ ]:
network_df = pd.concat([sub_com_network_df, comments_network_df], axis=0).reset_index(drop=True)
network_df # 34770 // 32781

,source,target,target_id,comment
0,throwawaytheist,jelindrael,1793g27,The Anti Planner\n\nIt's literally just a list...
1,No-Sea1173,jelindrael,1793g27,Don't know any books sorry. \n\nMaybe this wil...
2,Erpelpelle,Babybabyyyx,1793f20,"You can try to focus on your breathing, recogn..."
3,Cursed_Creative,YogurtclosetSoft3850,1792v8g,"Not to discourage you, but just for N=1, my ex..."
4,CaberTosser_,nilbogpt,1792p91,Yeah. Dopamine deficiency and depression can c...
...,...,...,...,...
34765,diamondaires,fae713,k8rhqcs,"Yes, I know they’re not legally required to do..."
34766,diamondaires,fae713,k8ri3c7,The reality is I should’ve gotten tested and d...
34767,diamondaires,Embarrassed_Tie_9346,k8r3r2t,I’ve only been able to find instances where it...
34768,fae713,diamondaires,k8w90xt,I can completely understand the frustration of...


In [ ]:
network_df.to_pickle("edgelist.pkl")

In [ ]:
network_df.to_csv("/content/comm557_network_df.csv")

In [ ]:
network_df.to_csv("/content/drive/MyDrive/comm557_network_df.csv")

In [ ]:
submissions_df.dtypes

id               object
full_id          object
author           object
title            object
text             object
url              object
is_self            bool
created_utc      object
edited           object
num_comments      int64
is_locked          bool
is_over_18         bool
score             int64
upvote_ratio    float64
link             object
dtype: object

In [ ]:
comments_df.dtypes

id               object
author           object
body             object
created_utc      object
edited           object
distinguished    object
replies           int64
submission_id    object
parent_id        object
is_submitter       bool
is_stickied        bool
score             int64
link             object
dtype: object

In [ ]:
network_df.dtypes

source       object
target       object
target_id    object
comment      object
dtype: object

In [ ]:
!pip install networkx
!pip install matplotlib

In [ ]:
import networkx as nx

import numpy as np
import nltk
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import urllib, json